# Fundaments

Calling LLM using Python

In [35]:
from llm_cookbook.utils.get_client import openai_client

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Call a normal LLM with chat completions API

In [2]:
sys_prompt = "You are a helpful assistant that knows about history"
input_ = "Tell me how to make a Omelette in 3 sentences"

message_stack = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": input_},
]

response = openai_client.chat.completions.create(messages=message_stack, model="gpt-4o")

In [4]:
# Check out the message content

print(response.choices[0].message.content)

Sure!  

1. Crack 2–3 eggs into a bowl, whisk them well with a pinch of salt and pepper, and optionally mix in ingredients like cheese, herbs, or chopped vegetables.  
2. Heat a nonstick pan over medium heat, add butter or oil, and pour the eggs into the pan, spreading them evenly.  
3. Cook until the edges set, fold or roll the omelette gently in half, and serve warm!  


In [5]:
# Understand the whole message

response.choices[0].message

ChatCompletionMessage(content='Sure!  \n\n1. Crack 2–3 eggs into a bowl, whisk them well with a pinch of salt and pepper, and optionally mix in ingredients like cheese, herbs, or chopped vegetables.  \n2. Heat a nonstick pan over medium heat, add butter or oil, and pour the eggs into the pan, spreading them evenly.  \n3. Cook until the edges set, fold or roll the omelette gently in half, and serve warm!  ', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

# Call a reasoning LLM with chat completions API

In [2]:
sys_prompt = "You are a helpful assistant that knows about history"
input_ = "Tell me how to make a Omelette in 3 sentences"

message_stack = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": input_},
]

response = openai_client.chat.completions.create(
    messages=message_stack, model="o4-mini"
)

In [3]:
# Check out the message content -> cannot see the reasoning part here

print(response.choices[0].message.content, "\n")

print("model:", response.model)

Crack two or three eggs into a bowl, season with a pinch of salt and pepper, and whisk until the mixture is smooth and slightly frothy. Heat a small knob of butter or a teaspoon of oil in a nonstick skillet over medium heat, pour in the eggs, and gently swirl the pan or stir with a spatula until the edges begin to set. Sprinkle in cheese, herbs, or other fillings, fold the omelette in half, let it cook for another 30 seconds, then slide it onto a plate. 

model: o4-mini


In [12]:
# Understand the whole message

response.choices[0].message

ChatCompletionMessage(content='Beat 2–3 eggs with a pinch of salt and pepper in a bowl, then heat a little butter in a nonstick pan over medium heat. Pour the eggs into the pan and, as they start to set, gently lift the edges so that the uncooked egg can flow underneath; when mostly cooked, add any fillings you enjoy and fold the omelette in half. This simple yet elegant dish, with its roots in French cuisine, has evolved over centuries into a versatile meal enjoyed around the world.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)

# Call a normal LLM with response API

In [5]:
sys_prompt = "You are a helpful assistant that knows about history"
input_ = "Tell me how to make a Omelette in 3 sentences"

message_stack = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": input_},
]

response = openai_client.responses.create(input=message_stack, model="gpt-4o")

In [6]:
# Check out the message content
print(response.output[0].content[0].text, "\n")

print(response.model)

To make an omelette, crack 2-3 eggs into a bowl, whisk them together with a pinch of salt and pepper, and optionally add milk for a fluffier texture. Heat a non-stick skillet with a small amount of butter or oil, then pour the egg mixture into the pan, tilting it gently to spread evenly. Cook on medium heat until the edges set, add desired fillings (like cheese, vegetables, or ham), fold the omelette in half, and serve warm. 

gpt-4o


# Call a reasoning LLM with response API

In [ ]:
sys_prompt = "You are a helpful assistant that knows about history"
input_ = "Tell me how to make a Omelette in 3 sentences"

message_stack = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": input_},
]

reasoning_response = openai_client.responses.create(
    input=message_stack,
    model="o4-mini",
    reasoning={
        "effort": "low",
        # "summary": "concise" #[not supported as org] auto, concise, or detailed (currently only supported with o4-mini and o3)
    },
    max_output_tokens=1000,
)

In [32]:
# Reasoning

print(f"Reasoning : {response.output[0]}\n")
# response.output[0].summary
# or
# response.reasoning.summary

# Output
response.output_text

# or
# print(response.output[1].content[0].text)

Reasoning : ResponseReasoningItem(id='rs_6824a055fdc08190844272a193e36c2b036e13c0529ff134', summary=[], type='reasoning', status=None)



'1. Beat two or three eggs in a bowl with a pinch of salt and pepper until yolks and whites are fully combined.  \n2. Heat a nonstick skillet over medium heat, add a pat of butter or a drizzle of oil, then pour in the eggs, swirling to coat the pan evenly.  \n3. As the edges set, gently lift them with a spatula to let uncooked egg flow underneath, then fold the omelette in half and slide it onto a plate when cooked through but still tender.'

In [33]:
print(response.model)

o4-mini


In [7]:
prompt = """
Write a bash script that takes a matrix represented as a string with
format '[1,2],[3,4],[5,6]' and prints the transpose in the same format.
"""

response = openai_client.responses.create(
    model="o4-mini",
    reasoning={"effort": "medium"},
    input=[{"role": "user", "content": prompt}],
)

print(response.output_text)

Here’s a self-contained bash script, `transpose.sh`, that takes a single argument of the form

  '[1,2],[3,4],[5,6]'

and prints its transpose in the same format:

  '[1,3,5],[2,4,6]'

```bash
#!/usr/bin/env bash
set -euo pipefail

if [[ $# -ne 1 ]]; then
  echo "Usage: $0 '[row1],[row2],...,[rowN]'" >&2
  exit 1
fi

input=$1

# strip the very first '[' and very last ']'
matrix=${input#\[}
matrix=${matrix%\]}

# replace every '],[' with a single '|' so we can split rows on '|'
rows_str=${matrix//],[/|}
IFS='|' read -r -a rows <<< "$rows_str"

# declare an associative array to hold the elements
declare -A m
R=${#rows[@]}    # number of rows
C=0              # will detect number of columns from first row

# parse each row into cols, store in m["i,j"]
for i in "${!rows[@]}"; do
  IFS=',' read -r -a cols <<< "${rows[i]}"
  # on first row, record column count; on later rows, sanity-check it
  if [[ $i -eq 0 ]]; then
    C=${#cols[@]}
  elif [[ ${#cols[@]} -ne $C ]]; then
    echo "Error: ro

In [14]:
prompt = """
Tell me how to make a Omelette in 3 sentences
"""

response = openai_client.responses.create(
    model="o4-mini",
    reasoning={"effort": "low"},
    input=[{"role": "user", "content": prompt}],
    max_output_tokens=1000,
)

print(response.output_text)

1. Beat two or three eggs in a bowl with a pinch of salt and pepper until yolks and whites are fully combined.  
2. Heat a nonstick skillet over medium heat, add a pat of butter or a drizzle of oil, then pour in the eggs, swirling to coat the pan evenly.  
3. As the edges set, gently lift them with a spatula to let uncooked egg flow underneath, then fold the omelette in half and slide it onto a plate when cooked through but still tender.
